## you/metz w266 project data exploration

In [147]:
import os
import pandas as pd
import numpy as np
import json
import pprint
import re
import nltk
## nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/evelyn_you/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
!ls -l

total 232
-rw-rw-r-- 1 evelyn_you evelyn_you    588 Jul 20 07:01  README.md
-rw-rw-r-- 1 evelyn_you evelyn_you 143730 Jul 20 07:01 'Screen Shot 2020-07-18 at 2.45.50 PM.png'
drwxrwxr-x 3 evelyn_you evelyn_you   4096 Jul 21 05:19  bin
drwxrwxr-x 6 evelyn_you evelyn_you   4096 Jul 21 21:57  data
-rw-rw-r-- 1 evelyn_you evelyn_you  39218 Jul 21 23:41  exploration.ipynb
drwxrwxr-x 9 evelyn_you evelyn_you   4096 Jul 21 20:16  goemotions
drwxrwxr-x 3 evelyn_you evelyn_you   4096 Jul 21 05:18  lib
drwxrwxr-x 2 evelyn_you evelyn_you   4096 Jul 20 07:01  model_trained
drwxrwxr-x 2 evelyn_you evelyn_you   4096 Jul 21 03:38  plots
-rw-rw-r-- 1 evelyn_you evelyn_you    289 Jul 21 05:18  pyvenv.cfg
-rw-rw-r-- 1 evelyn_you evelyn_you    715 Jul 20 07:01  run_commands
-rw-rw-r-- 1 evelyn_you evelyn_you    803 Jul 20 07:01  run_evaluation.sh
-rw-rw-r-- 1 evelyn_you evelyn_you    847 Jul 20 07:01  run_training.sh
-rw-rw-r-- 1 evelyn_you evelyn_you   1849 Jul 20 07:01  trm_common_commands.md


In [3]:
#data_dir="/home/travismetz/w266_project/data/empatheticdialogues"
data_dir="data"
data_folders=os.listdir(data_dir)
print (data_folders)


['processed', 'empatheticdialogues', '.ipynb_checkpoints', 'goemotions']


In [4]:
for data_folder in data_folders:
    print(data_folder,'\n',os.listdir(os.path.join(data_dir,data_folder)))
    print('-'*30)

processed 
 ['all_ed_data.csv', 'ed_for_goemotions.tsv']
------------------------------
empatheticdialogues 
 ['valid.csv', 'train.csv', 'test.csv']
------------------------------
.ipynb_checkpoints 
 []
------------------------------
goemotions 
 ['goemotions_3.csv', 'goemotions_2.csv', 'goemotions_1.csv']
------------------------------


### empatheticdialogues

In [5]:
data_folder='empatheticdialogues'
file_name='train.csv'
train=pd.read_csv(os.path.join(data_dir,data_folder,file_name),error_bad_lines=False)
#there are bad lines where columns got out of sequence so need error_bad_lines flag

b'Skipping line 2355: expected 8 fields, saw 10\nSkipping line 36628: expected 8 fields, saw 12\nSkipping line 49433: expected 8 fields, saw 10\nSkipping line 56957: expected 8 fields, saw 10\nSkipping line 65019: expected 8 fields, saw 10\n'


In [6]:
train.iloc[0]

conv_id                                               hit:0_conv:1
utterance_idx                                                    1
context                                                sentimental
prompt           I remember going to the fireworks with my best...
speaker_idx                                                      1
utterance        I remember going to see the fireworks with my ...
selfeval                                               5|5|5_2|2|5
tags                                                           NaN
Name: 0, dtype: object

In [7]:
train.shape

(76668, 8)

In [8]:
train.conv_id.unique()

array(['hit:0_conv:1', 'hit:1_conv:2', 'hit:1_conv:3', ...,
       'hit:12423_conv:24846', 'hit:12424_conv:24848',
       'hit:12424_conv:24849'], dtype=object)

In [9]:
train.head()

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5,NaN
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,0,Was this a friend you were in love with_comma_...,5|5|5_2|2|5,NaN
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5,NaN
3,hit:0_conv:1,4,sentimental,I remember going to the fireworks with my best...,0,Where has she gone?,5|5|5_2|2|5,NaN
4,hit:0_conv:1,5,sentimental,I remember going to the fireworks with my best...,1,We no longer talk.,5|5|5_2|2|5,NaN


In [10]:
train.context.value_counts()

surprised       3956
excited         2935
angry           2740
proud           2719
annoyed         2642
sad             2634
afraid          2510
lonely          2503
grateful        2487
terrified       2487
guilty          2463
anxious         2456
disgusted       2447
confident       2440
anticipating    2439
hopeful         2404
furious         2394
impressed       2381
disappointed    2356
nostalgic       2350
joyful          2346
jealous         2329
prepared        2292
content         2214
devastated      2192
embarrassed     2191
sentimental     2073
caring          2054
trusting        2001
ashamed         1976
apprehensive    1822
faithful        1435
Name: context, dtype: int64

In [11]:
def print_conversation(data,conv_id):
    conversation=data[data.conv_id==conv_id]
    print ('context:',conversation.context.iloc[0])
    print('prompt:',conversation.prompt.iloc[0])
    print('selfeval:',conversation.selfeval.iloc[0])
    print('\n')
    for utterance in conversation.utterance:
        print(utterance)

conv_id='hit:0_conv:1'
print_conversation(train,conv_id)

context: sentimental
prompt: I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world.
selfeval: 5|5|5_2|2|5


I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world.
Was this a friend you were in love with_comma_ or just a best friend?
This was a best friend. I miss her.
Where has she gone?
We no longer talk.
Oh was this something that happened because of an argument?


In [12]:
conversation_list=train.groupby('conv_id').last().reset_index().conv_id
conversation_list

0                hit:0_conv:1
1        hit:10000_conv:20000
2        hit:10000_conv:20001
3        hit:10001_conv:20002
4        hit:10002_conv:20004
                 ...         
17834     hit:9999_conv:19999
17835       hit:999_conv:1998
17836       hit:999_conv:1999
17837         hit:99_conv:198
17838           hit:9_conv:19
Name: conv_id, Length: 17839, dtype: object

In [13]:
for conv_id in conversation_list[0:10]:
    print_conversation(train,conv_id)
    print('\n\n','- '*30)

context: sentimental
prompt: I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world.
selfeval: 5|5|5_2|2|5


I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world.
Was this a friend you were in love with_comma_ or just a best friend?
This was a best friend. I miss her.
Where has she gone?
We no longer talk.
Oh was this something that happened because of an argument?


 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
context: surprised
prompt: My girlfriend got me a toad today! I was so shocked and happy!
selfeval: 5|5|5_5|5|5


My girlfriend got me a pet toad today!
Do you like toads?
I do! I was so happy when I opened the box and that fat mofo jumped out!
That was nice of your girlfriend_comma_ do you love her?


 - - - - - - - - - - - - - - - - - - - - - - -

### goemotions

In [52]:
data_folder = '../goemotions/data'
file_name = 'test.tsv'
goemotions_test = pd.read_csv(os.path.join(data_dir,data_folder,file_name), sep='\t', header=None)

In [131]:
print(goemotions_test.shape)
goemotions_test.columns = ["utterance", "emotion_labels", "speaker"]

## goemotions format
goemotions_test.head(20)

(5427, 3)


,utterance,emotion_labels,speaker
0,I’m really sorry about your situation :( Altho...,25,eecwqtt
1,It's wonderful because it's awful. At not with.,0,ed5f85d
2,"Kings fan here, good luck to you guys! Will be...",13,een27c3
3,"I didn't know that, thank you for teaching me ...",15,eelgwd1
4,They got bored from haunting earth for thousan...,27,eem5uti
5,Thank you for asking questions and recognizing...,15,ef2nq7i
6,You’re welcome,15,efdbh17
7,100%! Congrats on your job too!,15,ef0ec3b
8,I’m sorry to hear that friend :(. It’s for the...,24,ee8utmi
9,"Girlfriend weak as well, that jump was pathetic.",25,eeni74k


In [42]:
file_name = 'emotions.txt'
emotions = pd.read_csv(os.path.join(data_dir, data_folder, file_name), header=None)
emotions

,0
0,admiration
1,amusement
2,anger
3,annoyance
4,approval
5,caring
6,confusion
7,curiosity
8,desire
9,disappointment


In [50]:
file_name = 'sentiment_dict.json'
with open(os.path.join(data_dir, data_folder, file_name)) as json_data:
    sentiment_map = json.load(json_data)

In [51]:
pprint.pprint(sentiment_map)

{'ambiguous': ['realization', 'surprise', 'curiosity', 'confusion'],
 'negative': ['fear',
              'nervousness',
              'remorse',
              'embarrassment',
              'disappointment',
              'sadness',
              'grief',
              'disgust',
              'anger',
              'annoyance',
              'disapproval'],
 'positive': ['amusement',
              'excitement',
              'joy',
              'love',
              'desire',
              'optimism',
              'caring',
              'pride',
              'admiration',
              'gratitude',
              'relief',
              'approval']}


In [55]:
file_name = 'ekman_mapping.json'
with open(os.path.join(data_dir, data_folder, file_name)) as json_data:
    emotions_group = json.load(json_data)

In [56]:
pprint.pprint(emotions_group)

{'anger': ['anger', 'annoyance', 'disapproval'],
 'disgust': ['disgust'],
 'fear': ['fear', 'nervousness'],
 'joy': ['joy',
         'amusement',
         'approval',
         'excitement',
         'gratitude',
         'love',
         'optimism',
         'relief',
         'pride',
         'admiration',
         'desire',
         'caring'],
 'sadness': ['sadness', 'disappointment', 'embarrassment', 'grief', 'remorse'],
 'surprise': ['surprise', 'realization', 'confusion', 'curiosity']}


### process empatheticdialoges for goemotions classifier

In [16]:
data_folder = 'empatheticdialogues'
file_name = 'valid.csv'
all_ed_data = pd.read_csv(os.path.join(data_dir, data_folder, file_name), error_bad_lines=False, 
                          usecols=[0, 1, 2, 3, 4, 5, 6, 7])

In [17]:
print(all_ed_data.shape)
all_ed_data.head(10)

(12030, 8)


,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:3_conv:6,1,terrified,Today_comma_as i was leaving for work in the m...,6,Today_comma_as i was leaving for work in the m...,4|5|5_5|5|5,NaN
1,hit:3_conv:6,2,terrified,Today_comma_as i was leaving for work in the m...,7,Are you fine now?,4|5|5_5|5|5,NaN
2,hit:3_conv:6,3,terrified,Today_comma_as i was leaving for work in the m...,6,Yeah_comma_i'm doing alright now_comma_ but wi...,4|5|5_5|5|5,NaN
3,hit:3_conv:6,4,terrified,Today_comma_as i was leaving for work in the m...,7,Cool :) Is your car damaged a lot?,4|5|5_5|5|5,<IRREGULAR_COLON_FORMAT>
4,hit:3_conv:6,5,terrified,Today_comma_as i was leaving for work in the m...,6,The car was badly damaged_comma_i veered outsi...,4|5|5_5|5|5,NaN
5,hit:4_conv:9,1,surprised,I was walking through my hallway a few week ag...,8,A few weeks ago_comma_ I was walking through m...,5|5|5_3|5|5,NaN
6,hit:4_conv:9,2,surprised,I was walking through my hallway a few week ag...,4,That's funny_comma_ hope he didn't give you a ...,5|5|5_3|5|5,NaN
7,hit:4_conv:9,3,surprised,I was walking through my hallway a few week ag...,8,I may have let out a scream that will have him...,5|5|5_3|5|5,NaN
8,hit:4_conv:9,4,surprised,I was walking through my hallway a few week ag...,4,I would probably scream also.,5|5|5_3|5|5,NaN
9,hit:6_conv:12,1,excited,I'm overly excited today because will be flyin...,6,I'm overly excited because will be flying outs...,I have clicked the 'End Chat' button thank you...,NaN


In [18]:
## Write this out to a file for reference later
all_ed_data.to_csv ('data/processed/all_ed_data.csv', index = False, header = True)

In [19]:
## Process for goemotions
ed_for_goemotions = pd.DataFrame()
ed_for_goemotions["utterance"] = all_ed_data["utterance"].str.replace("_comma_", ", ", n=- 1, case=None, flags=0, regex=True)
ed_for_goemotions["utterance"] = ed_for_goemotions["utterance"].str.replace("  ", " ", n=- 1, case=None, flags=0, regex=True)
ed_for_goemotions["emotion_labels"] = 0 ## placeholder

In [20]:
ed_for_goemotions["speaker"] = all_ed_data["conv_id"] + "_" + all_ed_data["utterance_idx"].map(str) \
+ "_" + all_ed_data["speaker_idx"].map(str)

In [21]:
ed_for_goemotions.head()

,utterance,emotion_labels,speaker
0,"Today, as i was leaving for work in the mornin...",0,hit:3_conv:6_1_6
1,Are you fine now?,0,hit:3_conv:6_2_7
2,"Yeah, i'm doing alright now, but with minor in...",0,hit:3_conv:6_3_6
3,Cool :) Is your car damaged a lot?,0,hit:3_conv:6_4_7
4,"The car was badly damaged, i veered outside th...",0,hit:3_conv:6_5_6


In [22]:
## Write this out to a file for reference later
ed_for_goemotions.to_csv ('data/processed/ed_for_goemotions.tsv', sep="\t", index = False, header = False)

### process dailydialogues for goemotions classifier

Here are some explanations about the files:

1) dialogues_text.txt: The DailyDialog dataset which contains 11318 transcribed dialogues.

2) dialogues_topic.txt: Each line of dialogues_topic.txt corresponds to the topic of same line in dialogues_text.txt.
                        The representation of the topic number: {1: Ordinary Life, 2: School Life, 3: Culture & Education,
                        4: Attitude & Emotion, 5: Relationship, 6: Tourism , 7: Health, 8: Work, 9: Politics, 10: Finance}

3) dialogues_act.txt: Each line of dialogues_act.txt corresponds to the act of same line in dialogues_text.txt.
                      The representation of the act number: { 1: inform，2: question, 3: directive, 4: commissive }

4) dialogues_emotion.txt: Each line of dialogues_emotion.txt corresponds to the emotion of same line in dialogues_text.txt.
                          The representation of the emotion number: { 0: no emotion, 1: anger, 2: disgust, 3: fear, 4: happiness, 5: sadness, 6: surprise}

In [82]:
data_folder = 'dailydialogues'
file_name = 'dialogues_text.txt'
text_file = open(os.path.join(data_dir, data_folder, file_name), 'r')
dd_text = text_file.readlines()
text_file.close()

print(len(dd_text))
dd_text[:5]

13118


["The kitchen stinks . __eou__ I'll throw out the garbage . __eou__\n",
 'So Dick , how about getting some coffee for tonight ? __eou__ Coffee ? I don ’ t honestly like that kind of stuff . __eou__ Come on , you can at least try a little , besides your cigarette . __eou__ What ’ s wrong with that ? Cigarette is the thing I go crazy for . __eou__ Not for me , Dick . __eou__\n',
 'Are things still going badly with your houseguest ? __eou__ Getting worse . Now he ’ s eating me out of house and home . I ’ Ve tried talking to him but it all goes in one ear and out the other . He makes himself at home , which is fine . But what really gets me is that yesterday he walked into the living room in the raw and I had company over ! That was the last straw . __eou__ Leo , I really think you ’ re beating around the bush with this guy . I know he used to be your best friend in college , but I really think it ’ s time to lay down the law . __eou__ You ’ re right . Everything is probably going to come 

In [111]:
file_name = 'dialogues_emotion.txt'
text_file = open(os.path.join(data_dir, data_folder, file_name), 'r')
dd_emotion = text_file.readlines()
text_file.close()

len(dd_emotion)

13118

In [112]:
file_name = 'dialogues_act.txt'
text_file = open(os.path.join(data_dir, data_folder, file_name), 'r')
dd_act = text_file.readlines()
text_file.close()

len(dd_act)

13118

In [113]:
file_name = 'dialogues_topic.txt'
text_file = open(os.path.join(data_dir, data_folder, file_name), 'r')
dd_topic = text_file.readlines()
text_file.close()

len(dd_topic)

13118

In [123]:
## Find a way to extract each utterance in the conversations while keeping track of the conversations
convo_lengths = list()
utterances = list()

for x in range(len(dd_emotion)):
    convo_id = x+1
    convo_length = int((len(dd_emotion[x])-1)/2)
    convo_lengths.append(convo_length)
    
    convo = re.split(r'__eou__', dd_text[x])
    emo_tag = re.split(r' ', dd_emotion[x])
    act_tag = re.split(r' ', dd_act[x])
    topic = int(dd_topic[x])
    
    for y in range(convo_length):
        utterances.append([convo_id, topic, y+1, convo[y], act_tag[y], emo_tag[y]])

print("min exchange:", min(convo_lengths))
print("max exchange:", max(convo_lengths))

min exchange: 2
max exchange: 35


In [130]:
dd_data = pd.DataFrame(utterances)
dd_data.columns = ["convo_idx", "topic", "utterance_idx", "utterance", "act_label", "emotion_label"]
dd_data.head(10)

,convo_idx,topic,utterance_idx,utterance,act_label,emotion_label
0,1,1,1,The kitchen stinks .,3,2
1,1,1,2,I'll throw out the garbage .,4,0
2,2,1,1,"So Dick , how about getting some coffee for to...",3,4
3,2,1,2,Coffee ? I don ’ t honestly like that kind of...,4,2
4,2,1,3,"Come on , you can at least try a little , bes...",3,0
5,2,1,4,What ’ s wrong with that ? Cigarette is the t...,1,1
6,2,1,5,"Not for me , Dick .",1,0
7,3,1,1,Are things still going badly with your housegu...,2,0
8,3,1,2,Getting worse . Now he ’ s eating me out of h...,1,1
9,3,1,3,"Leo , I really think you ’ re beating around ...",3,0


In [217]:
## find ways to substitute names with [NAME] per goemotions
data_folder = 'other'
file_name = 'baby_names.csv'
baby_names = pd.read_csv(os.path.join(data_dir,data_folder,file_name))
baby_names.head()

,year,name,percent,sex
0,1880,John,0.081541,boy
1,1880,William,0.080511,boy
2,1880,James,0.050057,boy
3,1880,Charles,0.045167,boy
4,1880,George,0.043292,boy


In [219]:
file_name = 'last_names.csv'
last_names = pd.read_csv(os.path.join(data_dir,data_folder,file_name))
last_names.head()

FileNotFoundError: [Errno 2] File data/other/last_names.csv does not exist: 'data/other/last_names.csv'

In [186]:
## Get a list of 258,000 English names
eng_names = list(baby_names["name"])
eng_names.remove("Don") # gets confused with "Don't" when processing

print(eng_names[:100])
len(eng_names)

['John', 'William', 'James', 'Charles', 'George', 'Frank', 'Joseph', 'Thomas', 'Henry', 'Robert', 'Edward', 'Harry', 'Walter', 'Arthur', 'Fred', 'Albert', 'Samuel', 'David', 'Louis', 'Joe', 'Charlie', 'Clarence', 'Richard', 'Andrew', 'Daniel', 'Ernest', 'Will', 'Jesse', 'Oscar', 'Lewis', 'Peter', 'Benjamin', 'Frederick', 'Willie', 'Alfred', 'Sam', 'Roy', 'Herbert', 'Jacob', 'Tom', 'Elmer', 'Carl', 'Lee', 'Howard', 'Martin', 'Michael', 'Bert', 'Herman', 'Jim', 'Francis', 'Harvey', 'Earl', 'Eugene', 'Ralph', 'Ed', 'Claude', 'Edwin', 'Ben', 'Charley', 'Paul', 'Edgar', 'Isaac', 'Otto', 'Luther', 'Lawrence', 'Ira', 'Patrick', 'Guy', 'Oliver', 'Theodore', 'Hugh', 'Clyde', 'Alexander', 'August', 'Floyd', 'Homer', 'Jack', 'Leonard', 'Horace', 'Marion', 'Philip', 'Allen', 'Archie', 'Stephen', 'Chester', 'Willis', 'Raymond', 'Rufus', 'Warren', 'Jessie', 'Milton', 'Alex', 'Leo', 'Julius', 'Ray', 'Sidney', 'Bernard', 'Dan', 'Jerry', 'Calvin']


257999

In [180]:
def process_utterances(utterance, stop_words=eng_names):
    """Helper function to process the utterances into form accepted by goemotions"""
    
    """
    ## Replace names with a single token
    word_tokens = word_tokenize(utterance)
    filtered_sentence = ["[NAME]" if w in stop_words else w for w in word_tokens]
    processed = " ".join(filtered_sentence)
    """
    
    re.sub(utterance, r'{(\w\.?(-\w\.?)* )+}{(\w\.?(-\w\.?)*)+}', '[NAME]')
    
    ## Correct spacing
    processed = processed.replace(" ,", ",").replace(" .", ".").replace(" ?", "?").replace(" !", "!").replace(" ’ ", "’")

    return processed

example = dd_data.utterance.iloc[9]
process_utterance(example)

'[NAME], I really think you’re beating around the bush with this guy. I know he used to be your best friend in college, but I really think it’s time to lay down the law.'

In [215]:
utterance = "Leo Smith , I really think you ’ re beating around the bush with this guy . "
re.sub(r'([A-Z][a-z]*)([\\s\\\'-][A-Z][a-z]*)*', '[NAME]', utterance)

'[NAME] [NAME] , [NAME] really think you ’ re beating around the bush with this guy . '

In [202]:
?re.sub

In [182]:
utterance_list = dd_data.utterance
new_utterance = list()

for utterance in utterance_list:
    new_utterance.append(process_utterance(utterance))

new_utterance

['The kitchen stinks.',
 "I 'll throw out the garbage.",
 'So [NAME], how about getting some coffee for tonight?',
 'Coffee? I don’t honestly like that kind of stuff.',
 'Come on, you can at least try a little, besides your cigarette.',
 'What’s wrong with that? Cigarette is the thing I go crazy for.',
 'Not for me, [NAME].',
 'Are things still going badly with your houseguest?',
 'Getting worse. Now he’s eating me out of house and home. I’Ve tried talking to him but it all goes in one ear and out the other. He makes himself at home, which is fine. But what really gets me is that yesterday he walked into the living room in the raw and I had company over! That was the last straw.',
 '[NAME], I really think you’re beating around the bush with this guy. I know he used to be your best friend in college, but I really think it’s time to lay down the law.',
 'You’re right. Everything is probably going to come to a head tonight. I’ll keep you informed.',
 'Would you mind waiting a while?',
 'W